## ZERO-SHOT PROMPTING

In [1]:
!pip install langchain-groq

  Obtaining dependency information for langchain-groq from https://files.pythonhosted.org/packages/63/09/24e231c596d1892f75ad6700db326528a87d18fb20004d49f2b1afadc38f/langchain_groq-0.2.5-py3-none-any.whl.metadata
  Obtaining dependency information for langchain-core<1.0.0,>=0.3.42 from https://files.pythonhosted.org/packages/b6/8b/c41dbe55a502b71d84734be75c40d5a62fd6024dd8417c9ca2417e308782/langchain_core-0.3.45-py3-none-any.whl.metadata
  Obtaining dependency information for groq<1,>=0.4.1 from https://files.pythonhosted.org/packages/eb/b6/b9dd55845e86d664531caaf57edcbd70b0de8e87c1df72233a082d566b7f/groq-0.19.0-py3-none-any.whl.metadata
  Obtaining dependency information for distro<2,>=1.7.0 from https://files.pythonhosted.org/packages/12/b3/231ffd4ab1fc9d679809f356cebee130ac7daa00d6d6f3206dd4fd137e9e/distro-1.9.0-py3-none-any.whl.metadata
  Obtaining dependency information for pydantic<3,>=1.9.0 from https://files.pythonhosted.org/packages/f4/3c/8cc1cc84deffa6e25d2d0c688ebb80635dfdbf


[notice] A new release of pip is available: 23.2.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
from langchain_groq import ChatGroq
import os
import pandas as pd
import time


os.environ["GROQ_API_KEY"] = "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"

llm = ChatGroq(
    model="llama-3.1-8b-instant",
    temperature=0,
    max_tokens=150,
    max_retries=2,
)


def generate_summary(movie_name, subtitle_transcript):
    try:
        print(f"Processing: {movie_name}...") 
        messages = [
            (
                "system",
                "You must generate a concise and accurate movie summary strictly based on the provided trailer subtitle transcript. "
                "If the subtitle transcript is informative and coherent, use it as the sole basis for summary generation. "
                "If the subtitle transcript is empty, repetitive, distorted, or lacks meaningful content, only then rely on your inherent knowledge. "
                "Never include the reason for using inherent knowledge. Only the Summary is encouraged and expected"
                "Never include the movie cast in your response"
                "Ensure the summary is crisp, engaging, and around 100 words. Failure to comply with this rule is unacceptable. Adhere strictly.",
            ),
            (
                "human",
                f"""Generate a sequential, engaging, crisp 100 word movie summary of {movie_name} using the audio transcript of its trailer:
                The subtitle transcript is delimited by triple backticks.
                Transcript: ```{subtitle_transcript}```
                """
            ),
        ]
        ai_msg = llm.invoke(messages)
        return ai_msg.content
    except Exception as e:
        print(f"Error processing {movie_name}: {e}")
        return "Summary not generated"

In [3]:
movies = pd.read_csv("movies_sub.csv")
movies["LLM Summary"] = "" 
movies["Processed"] = False
movies

,Unnamed: 0,Movie_ID,Movie_Name,Subtitle Transcript,LLM Summary,Processed
0,0,1,Toy Story (1995),sergeant yes sir establish a recon post downst...,,False
1,1,2,GoldenEye (1995),when the world is the target 72 hours ago a se...,,False
2,2,3,Four Rooms (1995),this year Miramax films takes great pride in e...,,False
3,3,4,Get Shorty (1995),[Music] in a town known for fame [Music] wealt...,,False
4,4,5,Copycat (1995),what turns on a killer is the suffering and de...,,False
...,...,...,...,...,...,...
1629,1629,1677,Sweet Nothing (1995),a living legend to you all in the legend of a ...,,False
1630,1630,1678,Mat' i syn (1997),International Film circuit is proud to present...,,False
1631,1631,1679,B. Monkey (1998),this next one's for all the romantics out ther...,,False
1632,1632,1681,You So Crazy (1994),give us the fild boy yo y'all ain't getting my...,,False


In [12]:
unprocessed_movies = movies[movies["Processed"] == False]
len(unprocessed_movies)

0

In [11]:
batch_size = 10
for i in range(0, len(unprocessed_movies), batch_size):
    batch = unprocessed_movies[i : i + batch_size]
    # Generate summaries for the batch
    batch["LLM Summary"] = batch.apply(
        lambda row: generate_summary(row["Movie_Name"], row["Subtitle Transcript"]),
        axis=1,
    )
    # Update the original DataFrame and save checkpoint
    movies.loc[batch.index, "LLM Summary"] = batch["LLM Summary"]
    movies.loc[batch.index, "Processed"] = True
    movies.to_csv("movies_sub.csv", index=False)
    print(f"Checkpoint saved. Processed {i + batch_size} movies.")
    time.sleep(1)  # Optional: Add a small delay between batches

print("Processing completed!")

Processing: Bedknobs and Broomsticks (1971)...
Processing: Sound of Music, The (1965)...
Processing: Die Hard (1988)...
Processing: Funny Face (1957)...
Processing: Affair to Remember, An (1957)...
Processing: Little Lord Fauntleroy (1936)...
Processing: Inspector General, The (1949)...
Processing: Winnie the Pooh and the Blustery Day (1968)...
Processing: Hear My Song (1991)...
Processing: Mediterraneo (1991)...


C:\Users\typis\AppData\Local\Temp\ipykernel_7576\2692335480.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  batch["LLM Summary"] = batch.apply(


Checkpoint saved. Processed 10 movies.
Processing: Passion Fish (1992)...
Processing: Grateful Dead (1995)...
Processing: Eye for an Eye (1996)...
Processing: Fear (1996)...
Processing: Solo (1996)...
Processing: Substitute, The (1996)...
Processing: Heaven's Prisoners (1996)...
Processing: Trigger Effect, The (1996)...
Processing: Mother Night (1996)...
Processing: Dangerous Ground (1997)...


C:\Users\typis\AppData\Local\Temp\ipykernel_7576\2692335480.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  batch["LLM Summary"] = batch.apply(


Checkpoint saved. Processed 20 movies.
Processing: Maximum Risk (1996)...
Processing: Rich Man's Wife, The (1996)...
Processing: Shadow Conspiracy (1997)...
Processing: Blood & Wine (1997)...
Processing: Turbulence (1997)...
Processing: Underworld (1997)...
Processing: Beautician and the Beast, The (1997)...
Processing: Cats Don't Dance (1997)...
Processing: Anna Karenina (1997)...
Processing: Keys to Tulsa (1997)...


...

Checkpoint saved. Processed 680 movies.
Processing: Wedding Bell Blues (1996)...
Processing: MURDER and murder (1996)...
Processing: Tainted (1998)...
Processing: Further Gesture, A (1996)...
Processing: Kika (1993)...
Processing: Mirage (1995)...
Processing: Mamma Roma (1962)...
Processing: Sunchaser, The (1996)...
Processing: War at Home, The (1996)...
Processing: Sweet Nothing (1995)...


C:\Users\typis\AppData\Local\Temp\ipykernel_7576\2692335480.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  batch["LLM Summary"] = batch.apply(


Checkpoint saved. Processed 690 movies.
Processing: Mat' i syn (1997)...
Processing: B. Monkey (1998)...
Processing: You So Crazy (1994)...
Processing: Scream of Stone (Schrei aus Stein) (1991)...


C:\Users\typis\AppData\Local\Temp\ipykernel_7576\2692335480.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  batch["LLM Summary"] = batch.apply(


Checkpoint saved. Processed 700 movies.
Processing completed!


## IMDb SUMMARY GENERATION

In [27]:
#movies.drop('Processed', axis = 1, inplace = True)
movies['Processed'] = False
movies.insert(3, 'IMDb Summary', "")
movies

,Movie_ID,Movie_Name,Subtitle Transcript,IMDb Summary,Llama Summary Zero Shot,Processed
0,1,Toy Story (1995),sergeant yes sir establish a recon post downst...,,"In a world where toys come to life, a showdown...",False
1,2,GoldenEye (1995),when the world is the target 72 hours ago a se...,,"In a world on the brink of destruction, a secr...",False
2,3,Four Rooms (1995),this year Miramax films takes great pride in e...,,"At the mysterious Mon Senor hotel, a lone bell...",False
3,4,Get Shorty (1995),[Music] in a town known for fame [Music] wealt...,,"In the town of Hollywood, where fame and wealt...",False
4,5,Copycat (1995),what turns on a killer is the suffering and de...,,"In the gripping thriller Copycat, a cunning se...",False
...,...,...,...,...,...,...
1629,1677,Sweet Nothing (1995),a living legend to you all in the legend of a ...,,"In ""Sweet Nothing,"" Angelo Gazzetta, a 29-year...",False
1630,1678,Mat' i syn (1997),International Film circuit is proud to present...,,"In ""Mat' i syn,"" a breathtaking new film, a po...",False
1631,1679,B. Monkey (1998),this next one's for all the romantics out ther...,,"In the streets of London, a mysterious woman k...",False
1632,1681,You So Crazy (1994),give us the fild boy yo y'all ain't getting my...,,"In ""You So Crazy,"" a fiery comedian takes a st...",False


In [23]:
!pip install imdbpy


[notice] A new release of pip is available: 23.2.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip



  Obtaining dependency information for imdbpy from https://files.pythonhosted.org/packages/6f/3c/38555ae0ae26487eff082065a1f7b7a490f7f6e13ea249b8e82cfcc6eb31/IMDbPY-2022.7.9-py3-none-any.whl.metadata
  Obtaining dependency information for cinemagoer from https://files.pythonhosted.org/packages/a7/cc/959c7d74b7d6124852fc4741c154c8f50848f47360955f780636102ec711/cinemagoer-2023.5.1-py3-none-any.whl.metadata
  Obtaining dependency information for SQLAlchemy from https://files.pythonhosted.org/packages/97/c6/25ca068e38c29ed6be0fde2521888f19da923dbd58f5ff16af1b73ec9b58/sqlalchemy-2.0.39-cp312-cp312-win_amd64.whl.metadata
  Obtaining dependency information for lxml from https://files.pythonhosted.org/packages/2b/9c/8abe21585d20ef70ad9cec7562da4332b764ed69ec29b7389d23dfabcea0/lxml-5.3.1-cp312-cp312-win_amd64.whl.metadata
  Obtaining dependency information for greenlet!=0.4.17 from https://files.pythonhosted.org/packages/43/21/a5d9df1d21514883333fc86584c07c2b49ba7c602e670b174bd73cfc9c7f/greenl

In [35]:
from concurrent.futures import ThreadPoolExecutor
import pandas as pd
import time
from imdb import IMDb

ia = IMDb()

def fetch_movie_data(index, title):
    try:
        print(f"Processing: {index} --> {title}")
    
        if movies.loc[movies['Movie_ID'] == index, 'Completed'].values[0]:
            return
    
        search_results = ia.search_movie(title)
    
        if search_results:
            movie_id = search_results[0].movieID
            movie = ia.get_movie(movie_id)
            synopsis = movie.get('plot outline')
    
            movies.loc[movies['Movie_ID'] == index, ['IMDb Summary', 'Completed']] = [synopsis, True]
    
            if index % 50 == 0:
                movies.to_csv('movies_sub.csv', index=False)
                print(f"Progress saved at {index}")
    
        else:
            print(f"Movie '{title}' not found.")
    
        time.sleep(0.5)
        
    except:
        movies.loc[movies['Movie_ID'] == index, ['IMDb Summary', 'Completed']] = ['N/A', False]

        
with ThreadPoolExecutor(max_workers=5) as executor:
    for idx, row in movies.iterrows():
        executor.submit(fetch_movie_data, row['Movie_ID'], row['Movie_Name'])

# Final save
movies.to_csv('movies_sub.csv', index=False)
print("Final data saved to 'movies_sub.csv'")


Processing: 1 --> Toy Story (1995)
Processing: 2 --> GoldenEye (1995)
Processing: 3 --> Four Rooms (1995)
Processing: 4 --> Get Shorty (1995)
Processing: 5 --> Copycat (1995)
Processing: 6 --> Shanghai Triad (Yao a yao yao dao waipo qiao) (1995)
Processing: 7 --> Twelve Monkeys (1995)
Processing: 8 --> Babe (1995)
Processing: 9 --> Dead Man Walking (1995)
Processing: 10 --> Richard III (1995)
Processing: 11 --> Seven (Se7en) (1995)
Processing: 12 --> Usual Suspects, The (1995)
Processing: 13 --> Mighty Aphrodite (1995)
Processing: 14 --> Postino, Il (1994)
Processing: 15 --> Mr. Holland's Opus (1995)
Processing: 16 --> French Twist (Gazon maudit) (1995)
Processing: 17 --> From Dusk Till Dawn (1996)
Processing: 18 --> White Balloon, The (1995)
Processing: 19 --> Antonia's Line (1995)
Processing: 20 --> Angels and Insects (1995)
Processing: 21 --> Muppet Treasure Island (1996)
Processing: 22 --> Braveheart (1995)
Processing: 23 --> Taxi Driver (1976)
Processing: 24 --> Rumble in the Bron

2025-03-16 22:05:20,738 CRITICAL [imdbpy] C:\Users\typis\PycharmProjects\Classwork\venv310\Lib\site-packages\imdb\_exceptions.py:32: IMDbDataAccessError exception raised; args: ({'errcode': None, 'errmsg': 'None', 'url': 'https://www.imdb.com/find/?q=Apocalypse+Now+%281979%29&s=tt', 'proxy': '', 'exception type': 'IOError', 'original exception': <HTTPError 405: 'Not Allowed'>},); kwds: {}
Traceback (most recent call last):
  File "C:\Users\typis\PycharmProjects\Classwork\venv310\Lib\site-packages\imdb\parser\http\__init__.py", line 233, in retrieve_unicode
    response = uopener.open(url)
               ^^^^^^^^^^^^^^^^^
  File "C:\Users\typis\AppData\Local\Programs\Python\Python312\Lib\urllib\request.py", line 521, in open
    response = meth(req, response)
               ^^^^^^^^^^^^^^^^^^^
  File "C:\Users\typis\AppData\Local\Programs\Python\Python312\Lib\urllib\request.py", line 630, in http_response
    response = self.parent.error(
               ^^^^^^^^^^^^^^^^^^
  File "C:\Use

Processing: 181 --> Return of the Jedi (1983)
Processing: 182 --> GoodFellas (1990)
Processing: 184 --> Army of Darkness (1993)
Processing: 185 --> Psycho (1960)
Processing: 186 --> Blues Brothers, The (1980)
Processing: 187 --> Godfather: Part II, The (1974)
Processing: 188 --> Full Metal Jacket (1987)
Processing: 189 --> Grand Day Out, A (1992)
Processing: 190 --> Henry V (1989)
Processing: 191 --> Amadeus (1984)
Processing: 192 --> Raging Bull (1980)
Processing: 193 --> Right Stuff, The (1983)
Processing: 194 --> Sting, The (1973)
Processing: 195 --> Terminator, The (1984)
Processing: 196 --> Dead Poets Society (1989)
Processing: 197 --> Graduate, The (1967)
Processing: 198 --> Nikita (La Femme Nikita) (1990)
Processing: 199 --> Bridge on the River Kwai, The (1957)
Processing: 200 --> Shining, The (1980)
Processing: 201 --> Evil Dead II (1987)
Processing: 202 --> Groundhog Day (1993)


2025-03-16 22:05:22,287 CRITICAL [imdbpy] C:\Users\typis\PycharmProjects\Classwork\venv310\Lib\site-packages\imdb\_exceptions.py:32: IMDbDataAccessError exception raised; args: ({'errcode': None, 'errmsg': 'None', 'url': 'https://www.imdb.com/find/?q=Groundhog+Day+%281993%29&s=tt', 'proxy': '', 'exception type': 'IOError', 'original exception': <HTTPError 405: 'Not Allowed'>},); kwds: {}
Traceback (most recent call last):
  File "C:\Users\typis\PycharmProjects\Classwork\venv310\Lib\site-packages\imdb\parser\http\__init__.py", line 233, in retrieve_unicode
    response = uopener.open(url)
               ^^^^^^^^^^^^^^^^^
  File "C:\Users\typis\AppData\Local\Programs\Python\Python312\Lib\urllib\request.py", line 521, in open
    response = meth(req, response)
               ^^^^^^^^^^^^^^^^^^^
  File "C:\Users\typis\AppData\Local\Programs\Python\Python312\Lib\urllib\request.py", line 630, in http_response
    response = self.parent.error(
               ^^^^^^^^^^^^^^^^^^
  File "C:\User

Processing: 203 --> Unforgiven (1992)
Processing: 204 --> Back to the Future (1985)
Processing: 205 --> Patton (1970)
Processing: 206 --> Akira (1988)
Processing: 207 --> Cyrano de Bergerac (1990)
Processing: 208 --> Young Frankenstein (1974)
Processing: 209 --> This Is Spinal Tap (1984)
Processing: 210 --> Indiana Jones and the Last Crusade (1989)
Processing: 211 --> M*A*S*H (1970)
Processing: 212 --> Unbearable Lightness of Being, The (1988)
Processing: 213 --> Room with a View, A (1986)
Processing: 214 --> Pink Floyd - The Wall (1982)
Processing: 215 --> Field of Dreams (1989)
Processing: 216 --> When Harry Met Sally... (1989)
Processing: 217 --> Bram Stoker's Dracula (1992)
Processing: 218 --> Cape Fear (1991)
Processing: 219 --> Nightmare on Elm Street, A (1984)
Processing: 220 --> Mirror Has Two Faces, The (1996)
Processing: 221 --> Breaking the Waves (1996)
Processing: 222 --> Star Trek: First Contact (1996)
Processing: 223 --> Sling Blade (1996)
Processing: 224 --> Ridicule (19

...

Processing: 1069 --> Amateur (1994)
Processing: 1070 --> Living in Oblivion (1995)
Movie 'Until the End of the World (Bis ans Ende der Welt) (1991)' not found.
Processing: 1071 --> Party Girl (1995)
Processing: 1072 --> Pyromaniac's Love Story, A (1995)
Processing: 1073 --> Shallow Grave (1994)
Processing: 1074 --> Reality Bites (1994)
Processing: 1075 --> Man of No Importance, A (1994)
Processing: 1076 --> Pagemaster, The (1994)
Processing: 1077 --> Love and a .45 (1994)
Processing: 1078 --> Oliver & Company (1988)
Processing: 1079 --> Joe's Apartment (1996)
Processing: 1080 --> Celestial Clockwork (1994)
Movie 'Star Maker, The (Uomo delle stelle, L') (1995)' not found.
Processing: 1081 --> Curdled (1996)
Processing: 1082 --> Female Perversions (1996)
Processing: 1083 --> Albino Alligator (1996)
Processing: 1084 --> Anne Frank Remembered (1995)
Processing: 1085 --> Carried Away (1996)
Processing: 1086 --> It's My Party (1995)
Processing: 1087 --> Bloodsport 2 (1995)
Processing: 1088 -

2025-03-16 22:09:09,324 CRITICAL [imdbpy] C:\Users\typis\PycharmProjects\Classwork\venv310\Lib\site-packages\imdb\_exceptions.py:32: IMDbDataAccessError exception raised; args: ({'errcode': None, 'errmsg': 'None', 'url': 'https://www.imdb.com/find/?q=My+Family+%281995%29&s=tt', 'proxy': '', 'exception type': 'IOError', 'original exception': <HTTPError 405: 'Not Allowed'>},); kwds: {}
Traceback (most recent call last):
  File "C:\Users\typis\PycharmProjects\Classwork\venv310\Lib\site-packages\imdb\parser\http\__init__.py", line 233, in retrieve_unicode
    response = uopener.open(url)
               ^^^^^^^^^^^^^^^^^
  File "C:\Users\typis\AppData\Local\Programs\Python\Python312\Lib\urllib\request.py", line 521, in open
    response = meth(req, response)
               ^^^^^^^^^^^^^^^^^^^
  File "C:\Users\typis\AppData\Local\Programs\Python\Python312\Lib\urllib\request.py", line 630, in http_response
    response = self.parent.error(
               ^^^^^^^^^^^^^^^^^^
  File "C:\Users\ty

Processing: 1148 --> Tom & Viv (1994)
Processing: 1149 --> Walkabout (1971)
Processing: 1150 --> Last Dance (1996)
Processing: 1151 --> Original Gangstas (1996)
Processing: 1152 --> In Love and War (1996)
Processing: 1153 --> Backbeat (1993)
Processing: 1154 --> Alphaville (1965)
Processing: 1155 --> Rendezvous in Paris (Rendez-vous de Paris, Les) (1995)
Processing: 1156 --> Cyclo (1995)
Processing: 1157 --> Relic, The (1997)
Processing: 1158 --> Fille seule, La (A Single Girl) (1995)
Processing: 1159 --> Stalker (1979)
Processing: 1160 --> Love! Valour! Compassion! (1997)
Processing: 1161 --> Palookaville (1996)
Processing: 1162 --> Phat Beach (1996)
Processing: 1163 --> Portrait of a Lady, The (1996)
Processing: 1164 --> Zeus and Roxanne (1997)
Processing: 1165 --> Big Bully (1996)
Processing: 1166 --> Love & Human Remains (1993)
Processing: 1167 --> Sum of Us, The (1994)
Processing: 1168 --> Little Buddha (1993)
Processing: 1169 --> Fresh (1994)
Processing: 1170 --> Spanking the Mon

2025-03-16 22:09:27,443 CRITICAL [imdbpy] C:\Users\typis\PycharmProjects\Classwork\venv310\Lib\site-packages\imdb\_exceptions.py:32: IMDbDataAccessError exception raised; args: ({'errcode': None, 'errmsg': 'None', 'url': 'https://www.imdb.com/find/?q=Guilty+as+Sin+%281993%29&s=tt', 'proxy': '', 'exception type': 'IOError', 'original exception': <HTTPError 405: 'Not Allowed'>},); kwds: {}
Traceback (most recent call last):
  File "C:\Users\typis\PycharmProjects\Classwork\venv310\Lib\site-packages\imdb\parser\http\__init__.py", line 233, in retrieve_unicode
    response = uopener.open(url)
               ^^^^^^^^^^^^^^^^^
  File "C:\Users\typis\AppData\Local\Programs\Python\Python312\Lib\urllib\request.py", line 521, in open
    response = meth(req, response)
               ^^^^^^^^^^^^^^^^^^^
  File "C:\Users\typis\AppData\Local\Programs\Python\Python312\Lib\urllib\request.py", line 630, in http_response
    response = self.parent.error(
               ^^^^^^^^^^^^^^^^^^
  File "C:\User

Processing: 1216 --> Kissed (1996)
Processing: 1217 --> Assassins (1995)
Processing: 1218 --> Friday (1995)
Processing: 1219 --> Goofy Movie, A (1995)
Processing: 1220 --> Higher Learning (1995)
Processing: 1221 --> When a Man Loves a Woman (1994)
Processing: 1222 --> Judgment Night (1993)
Processing: 1223 --> King of the Hill (1993)
Processing: 1224 --> Scout, The (1994)
Processing: 1225 --> Angus (1995)
Processing: 1226 --> Night Falls on Manhattan (1997)
Processing: 1227 --> Awfully Big Adventure, An (1995)
Processing: 1228 --> Under Siege 2: Dark Territory (1995)
Processing: 1229 --> Poison Ivy II (1995)
Processing: 1230 --> Ready to Wear (Pret-A-Porter) (1994)
Processing: 1231 --> Marked for Death (1990)
Processing: 1232 --> Madonna: Truth or Dare (1991)
Processing: 1233 --> Nénette et Boni (1996)
Processing: 1234 --> Chairman of the Board (1998)
Processing: 1235 --> Big Bang Theory, The (1994)
Processing: 1236 --> Other Voices, Other Rooms (1997)
Processing: 1237 --> Twisted (199

In [40]:
movies.to_csv('movies_sub.csv', index = False)